In [1]:
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
import os
import cv2
import numpy as np
# Path of the pdf
PDF_file = "./data/leopold-nih-foia-anthony-fauci-emails.pdf"

In [ ]:
#making directories for intermediate statges
os.mkdir('./data/pdfsplits/')
os.mkdir('./data/jpgs/')

In [4]:
#splitting pdf
from PyPDF2 import PdfFileWriter, PdfFileReader

inputpdf = PdfFileReader(open(PDF_file, "rb"))

for i in range(inputpdf.numPages):
    output = PdfFileWriter()
    output.addPage(inputpdf.getPage(i))
    with open("./data/pdfsplits/document-page%s.pdf" % (i+1), "wb") as outputStream:
        output.write(outputStream)

In [5]:
import os
dir_list = os.listdir('./data/pdfsplits/')
len(dir_list)

3234

In [16]:
#converting to jpgs
image_counter = 1
for i in range(len(dir_list)):
    pages = convert_from_path('./data/pdfsplits/'+f'document-page{image_counter}.pdf', 500)
    for page in pages:
        filename = "page_"+str(image_counter)+".jpg"
        page.save('./data/jpgs/'+filename, 'JPEG')
    image_counter+=1

In [3]:
#couple cleaning functions that i found to process the image
import tempfile

import cv2
import numpy as np
from PIL import Image

IMAGE_SIZE = 1800
BINARY_THREHOLD = 180

def process_image_for_ocr(file_path):
    temp_filename = set_image_dpi(file_path)
    im_new = remove_noise_and_smooth(temp_filename)
    return im_new

def set_image_dpi(file_path):
    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = max(1, int(IMAGE_SIZE / length_x))
    size = factor * length_x, factor * width_y
    im_resized = im.resize(size, Image.ANTIALIAS)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(600, 600))
    return temp_filename

def image_smoothening(img):
    ret1, th1 = cv2.threshold(img, BINARY_THREHOLD, 255, cv2.THRESH_BINARY)
    ret2, th2 = cv2.threshold(th1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    blur = cv2.GaussianBlur(th2, (1, 1), 0)
    ret3, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return th3
def remove_noise_and_smooth(file_name):
    img = cv2.imread(file_name, 0)
    filtered = cv2.adaptiveThreshold(img.astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 41,
                                     3)
    kernel = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    img = image_smoothening(img)
    or_image = cv2.bitwise_or(img, closing)
    return or_image


In [7]:
#making the text files
for i in range(len(dir_list)):
    out_below = pytesseract.image_to_string(process_image_for_ocr(f'./data/jpgs/page_{i+1}.jpg'))
    with open(f'./data/text/page_{i+1}.txt','w') as f:
        f.write(out_below)